In [1]:
import pandas as pd
import psycopg2

In [2]:
df_banks = pd.read_csv("../data/banks.csv")
print(df_banks.head())
df= pd.read_csv("../data/processed/reviews_processed.csv")
print(df.head())

   bank_id                    bank_name
0        1  Commercial Bank of Ethiopia
1        2                   Awash Bank
2        3                  Dashen Bank
                              review_id  \
0  ba01d6b7-97fe-4376-b68c-817f397c00af   
1  bde60e8b-0b05-4c88-9f60-c14c18fa57b9   
2  fae55af8-1c5a-41f5-b6e8-55652afbf303   
3  e35ece9f-a9fb-4510-a9e3-8b8487a99d12   
4  fb9ed669-ea1d-4eb5-a336-634ccc59e265   

                                         review_text  rating review_date  \
0  I can't access the loan in the app. only it wo...       3  2025-11-28   
1     no one from all ethiopian bank keep it up nice       5  2025-11-28   
2                                      thenk you ✍️🙏       5  2025-11-26   
3                              i hope to be good app       5  2025-11-24   
4                                         amazing 😍😍       5  2025-11-23   

   review_year  review_month bank_code   bank_name           user_name  \
0         2025            11     Awash  Awash Bank

In [14]:
# -------------------------------------------------------
# Connect to PostgreSQL database
# -------------------------------------------------------


conn = psycopg2.connect(
    host="localhost",              
    database="bank_reviews", 
    user="admin",                 
    password="admin123"            
)

# Create a cursor object — used to execute SQL commands
cur = conn.cursor()

In [4]:
conn.rollback()


In [5]:
from psycopg2.extras import execute_values

rows = []
for row in df_banks.itertuples(index=False):
    bank_id = int(row.bank_id)
    bank_name = None if pd.isna(row.bank_name) else row.bank_name
    rows.append((bank_id, bank_name))

sql = """
INSERT INTO banks (bank_id, bank_name)
VALUES %s
ON CONFLICT (bank_id) DO NOTHING;
"""
execute_values(cur, sql, rows)
conn.commit()

In [6]:
for row in df.itertuples(index=False):
    print(row)  # inspect to see the attribute names


Pandas(review_id='ba01d6b7-97fe-4376-b68c-817f397c00af', review_text="I can't access the loan in the app. only it work in USSD", rating=3, review_date='2025-11-28', review_year=2025, review_month=11, bank_code='Awash', bank_name='Awash Bank', user_name='FAHMI SHEMSEDIN', thumbs_up=0, text_length=56, source='Google Play', sentiment_label='neutral', clean_text="i can't access the loan in the app. only it work in ussd", tokens='[\'i\', "can\'t", \'access\', \'the\', \'loan\', \'in\', \'the\', \'app.\', \'only\', \'it\', \'work\', \'in\', \'ussd\']', tokens_nostop='["can\'t", \'access\', \'loan\', \'app.\', \'work\', \'ussd\']', nouns="['loan', 'app', 'USSD']", tb_polarity=0.0, tb_subjectivity=1.0, tb_sentiment='neutral', vader_compound=0.0, vader_sentiment='neutral')
Pandas(review_id='bde60e8b-0b05-4c88-9f60-c14c18fa57b9', review_text='no one from all ethiopian bank keep it up nice', rating=5, review_date='2025-11-28', review_year=2025, review_month=11, bank_code='Awash', bank_name='Awash

In [7]:
# bank_name -> bank_id
bank_mapping = dict(zip(df_banks['bank_name'], df_banks['bank_id']))

In [8]:
rows = []
for row in df.itertuples(index=False):
    review_id = row.review_id
    bank_id = bank_mapping.get(row.bank_name)  # map bank_name to bank_id
    if bank_id is None:
        continue

    review_text = None if pd.isna(row.review_text) else row.review_text
    rating = float(row.rating) if not pd.isna(row.rating) else None
    review_date = row.review_date
    sentiment_score = row.vader_compound
    sentiment_label = row.vader_sentiment
    source = None if pd.isna(row.source) else row.source

    rows.append((
        review_id,
        bank_id,
        review_text,
        rating,
        review_date,
        sentiment_label,
        sentiment_score,
        source
    ))


In [12]:
print(df[['vader_sentiment', 'vader_compound']].head())


  vader_sentiment  vader_compound
0         neutral          0.0000
1        positive          0.1531
2         neutral          0.0000
3        positive          0.7003
4        positive          0.5859


In [9]:
rows[:5]

[('ba01d6b7-97fe-4376-b68c-817f397c00af',
  2,
  "I can't access the loan in the app. only it work in USSD",
  3.0,
  '2025-11-28',
  'neutral',
  0.0,
  'Google Play'),
 ('bde60e8b-0b05-4c88-9f60-c14c18fa57b9',
  2,
  'no one from all ethiopian bank keep it up nice',
  5.0,
  '2025-11-28',
  'positive',
  0.1531,
  'Google Play'),
 ('fae55af8-1c5a-41f5-b6e8-55652afbf303',
  2,
  'thenk you ✍️🙏',
  5.0,
  '2025-11-26',
  'neutral',
  0.0,
  'Google Play'),
 ('e35ece9f-a9fb-4510-a9e3-8b8487a99d12',
  2,
  'i hope to be good app',
  5.0,
  '2025-11-24',
  'positive',
  0.7003,
  'Google Play'),
 ('fb9ed669-ea1d-4eb5-a336-634ccc59e265',
  2,
  'amazing 😍😍',
  5.0,
  '2025-11-23',
  'positive',
  0.5859,
  'Google Play')]

In [10]:
from psycopg2.extras import execute_values

sql = """
INSERT INTO reviews
    (review_id, bank_id, review_text, rating, review_date, sentiment_label, sentiment_score, source)
VALUES %s
ON CONFLICT (review_id) DO NOTHING;
"""

execute_values(cur, sql, rows)
conn.commit()
cur.close()


In [11]:
df[["review_id","vader_compound","vader_sentiment"]].head()

,review_id,vader_compound,vader_sentiment
0,ba01d6b7-97fe-4376-b68c-817f397c00af,0.0000,neutral
1,bde60e8b-0b05-4c88-9f60-c14c18fa57b9,0.1531,positive
2,fae55af8-1c5a-41f5-b6e8-55652afbf303,0.0000,neutral
3,e35ece9f-a9fb-4510-a9e3-8b8487a99d12,0.7003,positive
4,fb9ed669-ea1d-4eb5-a336-634ccc59e265,0.5859,positive


In [15]:
from psycopg2.extras import execute_batch


cur = conn.cursor()

# Prepare data for update
update_rows = [
    (row.vader_sentiment, row.vader_compound, row.review_id)
    for row in df.itertuples()
]


In [16]:
sql = """
UPDATE reviews
SET sentiment_label = %s,
    sentiment_score = %s
WHERE review_id = %s;
"""

In [17]:
execute_batch(cur, sql, update_rows)
conn.commit()
cur.close()
conn.close()